In [12]:
#imports 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
#print('imports finished')

In [14]:
#read in data and preprocess it
df = pd.read_csv("Cancer_Data.csv")
df = df.drop('id',axis = 1)

#remove column of NaN that was tacked onto the end for some reason
df=df.dropna(axis=1,how='all')

#encodes data into numerical values
encoder = LabelEncoder()
df['diagnosis'] = encoder.fit_transform(df['diagnosis'])

#x and y data
y = df['diagnosis']
X = df.drop('diagnosis', axis = 1)

#scale data
scaler = StandardScaler()
X = scaler.fit_transform(X)

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [21]:
model = keras.Sequential([
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])
model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))

model.fit(X_train,y_train, epochs = 25, )

y_pred = model.predict(X_test)

model.evaluate(X_test, y_test)


Epoch 1/25
15/15 [==============================] - 1s 3ms/step - loss: 0.3108
Epoch 2/25
15/15 [==============================] - 0s 3ms/step - loss: 0.2212
Epoch 3/25
15/15 [==============================] - 0s 3ms/step - loss: 0.1930
Epoch 4/25
15/15 [==============================] - 0s 3ms/step - loss: 0.1612
Epoch 5/25
15/15 [==============================] - 0s 3ms/step - loss: 0.1514
Epoch 6/25
15/15 [==============================] - 0s 3ms/step - loss: 0.1377
Epoch 7/25
15/15 [==============================] - 0s 6ms/step - loss: 0.1303
Epoch 8/25
15/15 [==============================] - 0s 5ms/step - loss: 0.1277
Epoch 9/25
15/15 [==============================] - 0s 4ms/step - loss: 0.1123
Epoch 10/25
15/15 [==============================] - 0s 4ms/step - loss: 0.1105
Epoch 11/25
15/15 [==============================] - 0s 4ms/step - loss: 0.1028
Epoch 12/25
15/15 [==============================] - 0s 5ms/step - loss: 0.1039
Epoch 13/25
15/15 [==============================

0.14284633100032806